In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = (pd.read_csv('data/movies.dat',
                  delimiter="::",
                  header=None,
                  encoding='latin-1',
                  engine='python')
          .reset_index(drop=True)
         )

ratings = (pd.read_csv('data/ratings.dat',
                  delimiter="::",
                  header=None,
                  encoding='latin-1',
                  engine='python')
          .reset_index(drop=True)
         )

users = (pd.read_csv('data/users.dat',
                  delimiter="::",
                  header=None,
                  encoding='latin-1',
                  engine='python')
          .reset_index(drop=True)
         )

In [3]:
def rename_cols(df, cols):
    columns = dict(zip(range(0, len(cols)), cols))
    df.rename(columns=columns, inplace=True)

In [4]:
movies_cols = ['Movie-ID', 'Title', 'Genres']
users_cols = ['User-ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
ratings_cols = ['User-ID', 'Movie-ID', 'Rating', 'TimeStamp']

rename_cols(users, users_cols)
rename_cols(movies, movies_cols)
rename_cols(ratings, ratings_cols)

In [5]:
genres = []
[genres.extend(genre.split('|')) for genre in movies.Genres]
genres = set(genres)
genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [6]:
def get_dummies(value):
    if value.find(g) > -1:
        return 1
    return 0

In [7]:
movies['year'] = movies.Title.apply(lambda x: x[-5:-1])
movies['century'] = movies.year.apply(lambda x: x[:2] + str('00'))
movies['decade'] = movies.year.apply(lambda x: x[2:3] + str('0'))
movies['title'] = movies.Title.apply(lambda x: x[:-6].strip())
for g in genres:
    movies[g] = movies.Genres.apply(get_dummies)
movies.drop(['Title', 'Genres'], axis=1, inplace=True)

In [8]:
users['zip'] = users['Zip-code'].apply(lambda x: x[0] if len(x.split('-')) > 1 else x)
users.drop(['Zip-code'], axis=1, inplace=True)

In [9]:
users

,User-ID,Gender,Age,Occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [10]:
from datetime import datetime

dt_array = ratings.TimeStamp.apply(datetime.fromtimestamp)
ratings['hour'] = dt_array.apply(lambda x: x.hour)
ratings['day'] = dt_array.apply(lambda x: x.day)
ratings['month'] = dt_array.apply(lambda x: x.month)
ratings.drop(['TimeStamp'], axis=1, inplace=True)

In [11]:
ratings

,User-ID,Movie-ID,Rating,hour,day,month
0,1,1193,5,17,31,12
1,1,661,3,17,31,12
2,1,914,3,17,31,12
3,1,3408,4,17,31,12
4,1,2355,5,18,6,1
...,...,...,...,...,...,...
1000204,6040,1091,1,21,25,4
1000205,6040,1094,5,18,25,4
1000206,6040,562,5,18,25,4
1000207,6040,1096,4,21,25,4


In [12]:
results = pd.merge(users, pd.merge(movies, ratings, on='Movie-ID'), on='User-ID')
results

,User-ID,Gender,Age,Occupation,zip,Movie-ID,year,century,decade,title,...,Western,Thriller,Romance,Musical,Action,Animation,Rating,hour,day,month
0,1,F,1,10,48067,1,1995,1900,90,Toy Story,...,0,0,0,0,0,1,5,18,6,1
1,1,F,1,10,48067,48,1995,1900,90,Pocahontas,...,0,0,1,1,0,1,5,18,6,1
2,1,F,1,10,48067,150,1995,1900,90,Apollo 13,...,0,0,0,0,0,0,5,17,31,12
3,1,F,1,10,48067,260,1977,1900,70,Star Wars: Episode IV - A New Hope,...,0,0,0,0,1,0,4,17,31,12
4,1,F,1,10,48067,527,1993,1900,90,Schindler's List,...,0,0,0,0,0,0,5,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,M,25,6,11106,3683,1984,1900,80,Blood Simple,...,0,0,0,0,0,0,4,3,14,6
1000205,6040,M,25,6,11106,3703,1981,1900,80,Mad Max 2 (a.k.a. The Road Warrior),...,0,0,0,0,1,0,4,18,28,7
1000206,6040,M,25,6,11106,3735,1973,1900,70,Serpico,...,0,0,0,0,0,0,4,3,14,6
1000207,6040,M,25,6,11106,3751,2000,2000,00,Chicken Run,...,0,0,0,0,0,1,4,18,28,7
